# Aggregate scores and create RAG ratings

Aggregate the synthetic person-level standard survey responses to find the mean scores by area, then generate RAG ratings based on the scores.

## Set-up

### Packages and file paths

In [1]:
# Import required packages
from dataclasses import dataclass
import numpy as np
import os
import pandas as pd

# Import functions from our dashboard package
from kailo_beewell_dashboard.create_and_aggregate_data import (
    aggregate_scores, results_by_site_and_group, create_rag_ratings)

In [2]:
# File paths
@dataclass(frozen=True)
class Paths:
    '''Stores paths to data and files'''
    survey = '../data/survey_data'
    synthetic_data = 'standard_synthetic_data_raw_msoa.csv'
    aggregate_score = 'standard_area_aggregate_scores_rag.csv'


paths = Paths()

### Import raw data

In [3]:
data = pd.read_csv(os.path.join(paths.survey, paths.synthetic_data))
data.head()

,gender,transgender,sexual_orientation,neurodivergent,birth_parent1,birth_parent2,birth_you,birth_you_age,autonomy_pressure,autonomy_express,...,peer_talk_helpful_lab,peer_talk_if_lab,accept_peer_lab,year_group_lab,fsm_lab,sen_lab,ethnicity_lab,english_additional_lab,school_lab,msoa
0,4.0,2.0,6.0,3.0,2.0,NaN,2.0,1.0,2.0,5.0,...,Somewhat helpful,NaN,Not at all,Year 10,Non-FSM,Non-SEN,Ethnic minority,NaN,School E,Torridge 001
1,NaN,2.0,1.0,3.0,3.0,2.0,3.0,8.0,4.0,2.0,...,NaN,Very uncomfortable,Slightly,Year 10,Non-FSM,Non-SEN,Ethnic minority,No,School D,North Devon 001
2,2.0,3.0,4.0,1.0,1.0,1.0,1.0,1.0,5.0,4.0,...,NaN,Very comfortable,Not at all,Year 10,Non-FSM,Non-SEN,White British,No,School E,North Devon 007
3,2.0,5.0,5.0,NaN,2.0,2.0,NaN,3.0,1.0,2.0,...,NaN,Uncomfortable,NaN,Year 10,Non-FSM,Non-SEN,White British,No,School G,North Devon 006
4,5.0,3.0,4.0,1.0,1.0,3.0,3.0,2.0,5.0,2.0,...,Somewhat helpful,NaN,Not at all,NaN,Non-FSM,Non-SEN,White British,Yes,School B,Torridge 003


## Find mean score by MSOA

In [4]:
# Create version where every question has mean NaN and count 0, to use when a
# school has no pupils of a particular subgroup (i.e. no-one in certain
# FSM/SEN/gender/year)
no_pupils = aggregate_scores(data)
no_pupils['mean'] = np.nan
no_pupils['count'] = 0
no_pupils.head()

,variable,mean,count
0,autonomy_score,NaN,0
1,life_satisfaction_score,NaN,0
2,optimism_score,NaN,0
3,wellbeing_score,NaN,0
4,esteem_score,NaN,0


In [5]:
# Aggregate for each of the possible schools and pupils groups
agg = results_by_site_and_group(
    data=data, agg_func=aggregate_scores, no_pupils=no_pupils,
    group_type='none', site_col='msoa')

# Hide results when n<10
agg.loc[agg['count'] < 10, ['mean', 'count']] = np.nan

agg.head()

,variable,mean,count,msoa
0,autonomy_score,18.117647,17.0,North Devon 001
1,life_satisfaction_score,6.000000,27.0,North Devon 001
2,optimism_score,12.400000,30.0,North Devon 001
3,wellbeing_score,23.058824,17.0,North Devon 001
4,esteem_score,11.722222,18.0,North Devon 001


In [6]:
# Show the data types and presence of null
types = agg.dtypes
null = agg.isnull().any()
pd.DataFrame([types, null]).T

,0,1
variable,object,False
mean,float64,True
count,float64,True
msoa,object,False


## Calculate weighted means and standard deviations

In [7]:
rag = create_rag_ratings(agg)
rag.head()

,variable,mean,count,msoa,total_pupils,group_n,group_wt_mean,group_wt_std,lower,upper,rag
0,autonomy_score,18.117647,17.0,North Devon 001,475.0,23.0,17.791579,0.737471,17.054108,18.529049,average
1,life_satisfaction_score,6.000000,27.0,North Devon 001,702.0,23.0,5.055556,0.510489,4.545067,5.566045,above
2,optimism_score,12.400000,30.0,North Devon 001,707.0,23.0,12.066478,0.420813,11.645666,12.487291,average
3,wellbeing_score,23.058824,17.0,North Devon 001,528.0,23.0,21.022727,0.930389,20.092339,21.953116,above
4,esteem_score,11.722222,18.0,North Devon 001,460.0,22.0,12.828261,0.580838,12.247423,13.409099,below


## Save results

In [8]:
agg.to_csv(os.path.join(paths.survey, paths.aggregate_score),
           index=False, na_rep='NULL')